# Project 3: Wrangle Open Street Map Data
## Data Wrangling for Auditing and Cleaning
**Location: Miami, Florida**

In [1]:
"""
Auditing and cleaning the data
"""
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
import csv

# File from OpenStreetMap.org for Miami, Florida, USA. Size: 353.9 MB.
OSMFILE = "miami_florida.osm"

# Audit regex objects to parse out directions and street types and assess the validity of zip codes
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE) # Street types grabs the last word from the street name
direction_re = re.compile(r'^[NSEW]\S*\s', re.IGNORECASE) # Direction grabs the first word/letter from the street name
zip_re = re.compile(r'^33\d{3}($|(-\d{4})$)', re.IGNORECASE) # Zip looks for 5-digits starting with 33, allows for +4

# List of common street names in Miami and elsewhere in South Florida
expected_st = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Circle", "Run", "Terrace", "Highway", "Trace", "Plaza",
            "Causeway", "Path", "Isle", "Lake", "Row", "Crescent", "Manor"]

# Because the list of possible street types is so large, it was easier to maintain it in a separate CSV file
# generate_standardized_street_mapping is a function that reads in this file for mapping purposes during the audit
# This is the function that reads in the street type mapping CSV file and stores it in a dictionary called "mappings"
def generate_standardized_street_mapping(street_type_file):
    mappings = {}
    fieldnames = ["abbreviation", "preferred"]
    with open(street_type_file, "rU") as csvfile:
        reader = csv.DictReader(csvfile, fieldnames=fieldnames)
        for row in reader:
            mappings[row["abbreviation"]] = row["preferred"]
    csvfile.close()
    return mappings
                      
STREET_TYPE_CSV = "standardized_street_types.csv"
st_mapping = generate_standardized_street_mapping(STREET_TYPE_CSV)    


# List of common directions assigned to streets in Miami and elsewhere in South Florida
#expected_dir = ["Northwest", "Northeast", "Southwest", "Southeast", "North", "South", "East", "West"]
expected_dir = []

# After iterating through the audit, I discovered the following non-standard directional names need to be changed
dir_mapping = { "NW" : "Northwest",
                "NE" : "Northeast",
                "SW" : "Southwest",
                "SE" : "Southeast",
                "N.W." : "Northwest",
                "Norhwest" : "Northwest",
                "sw" : "Southwest",
                "southwest" : "Southwest",
                "S.W." : "Southwest",
                "N." : "North",
                "S." : "South",
                "E." : "East",
                "W." : "West",
                "N" : "North",
                "S" : "South",
                "E" : "East",
                "W" : "West",
                "wellington" : "Wellington" # Found this during the audit and threw it in here to capitalize the W               
                 }

# In going through the list of bad zip codes generated by my audit, I found a few that needed to be mapped 
# based on the results of a few manual Google map searches
zip_mapping = { "11890" : "33181",
               "3331" : "33313",
               "3447" : "33407",
               "(561) 795-4333" : "33411",
               "0" : "33326",
               u'361-0529\u200e' : "33431",
               "FL" : "33185"
              }

# Two global sets to store the address parts (street type and direction) that the audit picked up, but 
# were not changed
dir_not_found = set()
st_not_found = set()


# This is a generic function to check a specific part of the street name, whether it is the direction on the 
# front of the street name or the street type at the end. This is called from audit_street_type and audit_direction
def audit_address_part(regex_obj, types, street_name, expected):
    a = regex_obj.search(street_name)
    if a:
        address_part = a.group().strip()
        if address_part not in expected:
            types[address_part].add(street_name)
            

# This function calls audit_address_part specifically to audit a street type            
def audit_street_type(street_types, street_name):
    audit_address_part(street_type_re, street_types, street_name, expected_st)
    

# This function calls audit_address_part specifically to audit a direction name    
def audit_direction(dir_types, street_name):
    audit_address_part(direction_re, dir_types, street_name, expected_dir)
            

# This function determines if an XML tag is a street name
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


# This function determines if an XML tag is a state abbreviation
def is_state(elem):
    return (elem.attrib['k'] == "addr:state")


# This function loops through all of the street names in the OSM file and outputs dictionaries for all
# possible street types and directions, coupled with an array of all streets names found with those characteristics
def audit_street_names(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    dir_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
                    audit_direction(dir_types, tag.attrib['v'])
    osm_file.close()
    return street_types, dir_types


# This function determins if an XML tag refers to a zip code (aka "postcode")
def is_zip(elem):
    return (elem.attrib['k'] == "addr:postcode")


# This function takes a zip_code and a "bad_zips" list as inputs. If the zip code does not match the 
# regular expression for proper zip code formatting, the zip code is written to the bad_zips list
def audit_zip(bad_zips, zip_code):
    z = zip_re.search(zip_code)
    if not z:
        bad_zips.add(zip_code)


# This function runs through the entire OSM file looking for any "bad zips" as determined by audit_zip above, and 
# returns the entire list of bad zip codes that it finds
def audit_zips(osmfile):
    osm_file = open(osmfile, "r")
    bad_zips = set()
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zip(tag):
                    audit_zip(bad_zips, tag.attrib['v'])
    osm_file.close()
    return bad_zips


# This function is a generic function that reads either the direction (at the front of the string) or the 
# street type (and the end of the string), determines if it is erroneous, and if so, corrects the value of 
# that part of the string before returning the whole string
def update_address_part(regex_obj, name, mapping, category):
    
    # This function behaves slightly differently if we are dealing with a street type versus a direction.
    # The replacement strings needs a leading space for a street type and a trailing space for a direction.
    # Also, the global sets that store the items that were "not found" in the mapping have different names.
    if category == "street_type":
        expected = expected_st
        not_found_set = st_not_found # Removed for the actual update process
    elif category == "direction":
        expected = expected_dir
        not_found_set = dir_not_found # Removed for the actual update process
    
    # First, search for a string that matches the regex expression in the street name
    address_part = regex_obj.search(name)
    
    # Grab the string that is found and remove any whitespace
    if address_part:
        a = address_part.group().strip()
        
        # Check to if an unexpected string that we found is in the mapping. If so, replace it with the value 
        # found at that key in the mapping.
        if a not in expected:
            if a in mapping.keys():
                new_address_part = mapping[a]
                if category == "street_type": 
                    start, middle, finish = name.rpartition(" ") 
                    name = start + middle + new_address_part
                elif category == "direction":
                    start, middle, finish = name.partition(" ")
                    name = new_address_part + middle + finish
                
            # Otherwise, keep track of the address parts that were not mapped for auditing purposes.
            # (Removed for the actual update process)
            else:
                not_found_set.add(a)
            
    return name

# This function calls update_address_part twice, once for the street type and once for the direction and 
# returns the corrected name
def update_street_name(name, st_mapping, dir_mapping):
    
    # For a given street name, first update the street type according to the street type mapping
    name = update_address_part(street_type_re, name, st_mapping, "street_type")
    
    # Then update the direction according to the direction mapping
    name = update_address_part(direction_re, name, dir_mapping, "direction")
    
    # When done, return the street name with its street type and/or direction fixed
    return name


# This function works on improperly formatted zip codes to assign mapping values, strip out spaces, or strip out
# inappropriate state abbreviations
def update_zip(zip_code, zip_mapping):
    z = zip_re.search(zip_code)
    if not z:
        if zip_code in zip_mapping.keys():
            zip_code = zip_mapping[zip_code]
        elif zip_re.search(zip_code.strip()):
            zip_code = zip_code.strip()
        else:
            zip_code = zip_code.strip("FL-").strip("Fl").strip("FL").strip()
    return zip_code

st_types, dir_types = audit_street_names(OSMFILE)
#pprint.pprint(st_types)


# In order to determine which naming convention to use, I needed to define an "expected direction" array. 
# I started looking at the count information from auditing the directions
for d in dir_types:
    if d[0] in "NESWnesw":
        print d, len(dir_types[d])


Sandalwood 1
Northeast 28
Sunflower 1
Waterbrook 1
Nandina 1
Southfields 1
Saint 3
Eastlake 1
Whitaker 1
Worth 1
Entrada 1
Southwest 53
Windmill 2
Stanton 2
Equestrian 2
Sacramento 1
Elm 1
Stallion 1
Seabreeze 3
Nob 1
Simonton 1
Emerald 2
Sequoia 1
World 1
southwest 1
Equine 1
Saratoga 1
Newport 1
Elderberry 1
Southern 1
Shadyside 1
S.W. 1
Ensenada 1
Scripps 1
San 6
Savannah 1
Sevilla 1
Washington 2
Spoonbill 1
Seville 1
Nighthawk 2
Springside 2
Westwood 1
Windsor 1
Sunrise 1
Whitehead 1
Willow 4
West 26
Silks 1
Northpark 1
State 3
Sportsplex 1
Northwest 64
Seminole 4
Seagrape 1
East 15
Norhwest 1
Schooner 1
Wiles 1
E. 2
Swansea 1
Water 3
Southeast 16
N.W. 1
Stillwater 1
Stone 1
Northumberland 1
Silverbell 1
S 13
Slippery 1
W 13
Seneca 1
Sorrento 1
Westgate 1
Sunnyland 1
Sand 2
Sandpiper 1
SE 1
Northlake 1
Smoke 1
Waterview 3
Sailboat 1
Winterberry 1
Saddlebrook 1
Stonebrook 1
S. 1
Winfield 1
Waterford 1
Sirius 1
Staghorn 1
North 23
Waters 2
Sunny 2
NE 5
Sabal 3
Eller 1
Woodfall 1
West

In [2]:
# From there, I manually picked out all possible direction names and put them in a list called "direction_names"
direction_names = [
    "Northwest", "Northeast", "Southwest", "Southeast",
    "northwest", "northeast", "southwest", "southeast",
    "N.W.", "N.E.", "S.W.", "S.E.",
    "NW", "NE", "SW", "SE",
    "nw", "ne", "sw", "se",
    "North", "South", "East", "West",
    "N", "S", "E", "W",
    "N.", "S.", "E.", "W."
]

# Using that array helped me isolate the counts of just the direction names
# Based on those results (below), I chose to use the full direction name instead of the abbreviation
for d in direction_names:
    if d in dir_types:
        print d, len(dir_types[d])

Northwest 64
Northeast 28
Southwest 53
Southeast 16
southwest 1
N.W. 1
S.W. 1
NW 19
NE 5
SW 45
SE 1
sw 1
North 23
South 27
East 15
West 26
N 13
S 13
E 11
W 13
N. 3
S. 1
E. 2


In [3]:
st_types, dir_types = audit_street_names(OSMFILE)

print "Examples of some of the street names that were considered for cleaning and the result of cleaning"
print "(truncated for the sake of brevity):"
for st_type, ways in st_types.iteritems():
    i = 0
    for name in ways:
        better_name = update_street_name(name, st_mapping, dir_mapping)
        if i % 4800 == 0:
            print name, "=>", better_name
            i = i + 1

print "---"
print "Address last words that are potentially street names but were not fixed:"
print st_not_found

Examples of some of the street names that were considered for cleaning and the result of cleaning
(truncated for the sake of brevity):
Sunset Springs Spgs => Sunset Springs Springs
Capistrano => Capistrano
Country Isles Rd => Country Isles Road
East Las Olas BLVD => East Las Olas Boulevard
Osprey Bnd => Osprey Bend
US Highway 1 => US Highway 1
Hwy 27 => Hwy 27
Alhambra Cirlce => Alhambra Circle
Sacramento => Sacramento
E McNab RD => East McNab Road
Atlanta => Atlanta
Sabal Trl => Sabal Trail
Augusta => Augusta
E Merion => East Merion
Via Poinciana => Via Poinciana
826 => 826
San Luis Rey => San Luis Rey
Water Pt => Water Point
Marina Bay Drive West => Marina Bay Drive West
Country Club Cres => Country Club Crescent
Birkdale => Birkdale
184th street => 184th Street
Broken Sound Parkway Northwest => Broken Sound Parkway Northwest
Oakmont => Oakmont
3196 N. Jog Rd. => 3196 N. Jog Road
11860 W State Road 84 => 11860 W State Road 84
Fox Holw => Fox Hollow
Biscayne Blvd Ste 918 => Biscayne B

In [4]:
# List of common directions assigned to streets in Miami and elsewhere in South Florida
expected_dir = ["Northwest", "Northeast", "Southwest", "Southeast", "North", "South", "East", "West"]
dir_not_found = set()
st_types, dir_types = audit_street_names(OSMFILE)
for dir_type, ways in dir_types.iteritems():
    for name in ways:
        better_name = update_street_name(name, st_mapping, dir_mapping)
#        print name, "=>", better_name

zip_codes = audit_zips(OSMFILE)
    
for zip in zip_codes:
    better_zip = update_zip(zip, zip_mapping)
    print zip, "=>", better_zip
    
print "---"
print "Address first words that are potentially directions but were not fixed:"
print dir_not_found

33314  => 33314
FL 33487-3536 => 33487-3536
FL 33431 => 33431
FL 33433 => 33433
FL 33026 => 33026
361-0529‎ => 33431
FL 33033 => 33033
FL 33126 => 33126
Fl 33186 => 33186
FL  33351 => 33351
FL 33431-4403 => 33431-4403
FL => 33185
FL 33166 => 33166
0 => 33326
FL 33312 => 33312
FL-33140 => 33140
FL 33134 => 33134
FL 33016 => 33016
3331 => 33313
FL 33012 => 33012
11890 => 33181
FL 33140 => 33140
FL33401 => 33401
(561) 795-4333 => 33411
FL 33322 => 33322
FL-33139 => 33139
3447 => 33407
FL 33131 => 33131
---
Address first words that are potentially directions but were not fixed:
set(['Sandalwood', 'Southfields', 'Sunflower', 'Waterbrook', 'Nandina', 'Saint', 'Eastlake', 'Whitaker', 'Worth', 'Entrada', 'Windmill', 'Stanton', 'Equestrian', 'Sacramento', 'Waterview', 'Stallion', 'Seabreeze', 'Nob', 'Simonton', 'Emerald', 'Sequoia', 'World', 'Equine', 'Saratoga', 'Newport', 'Elderberry', 'Shadyside', 'Ensenada', 'San', 'Savannah', 'Sevilla', 'Washington', 'Spoonbill', 'Seville', 'Nighthawk', 'S

In [5]:
def update_address_part(regex_obj, name, mapping, category):
    
    # This function behaves slightly differently if we are dealing with a street type versus a direction.
    # The replacement strings needs a leading space for a street type and a trailing space for a direction.
    # Also, the global sets that store the items that were "not found" in the mapping have different names.
    if category == "street_type":
        expected = expected_st
        #not_found_set = st_not_found # REMOVED FOR THE ACTUAL UPDATE PROCESS
    elif category == "direction":
        expected = expected_dir
        #not_found_set = dir_not_found # Removed for the actual update process
    
    # First, search for a string that matches the regex expression in the street name
    address_part = regex_obj.search(name)
    
    # Grab the string that is found and remove any whitespace
    if address_part:
        a = address_part.group().strip()
        
        # Check to if an unexpected string that we found is in the mapping. If so, replace it with the value 
        # found at that key in the mapping.
        if a not in expected:
            if a in mapping.keys():
                new_address_part = mapping[a]
                if category == "street_type":
                    start, middle, finish = name.rpartition(" ")
                    name = start + middle + new_address_part
                elif category == "direction":
                    start, middle, finish = name.partition(" ")
                    name = new_address_part + middle + finish
                
            # Otherwise, keep track of the address parts that were not mapped for auditing purposes.
            # (Removed for the actual update process)
            # else:
            #     not_found_set.add(a)
            
    return name

In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json
"""
Your task is to wrangle the data, transform the shape of the data, and export a list 
of dictionaries to a JSON file to be imported into MongoDB.

You have to complete the function 'shape_element'.
We have provided a function that will parse the map file, and call the function with the element
as an argument. You should return a dictionary, containing the shaped data for that element.
We have also provided a way to save the data in a file, so that you could use
mongoimport later on to import the shaped data into MongoDB. 

If you are using this code in your final project, you are strongly encouraged to use the code 
from previous exercise to update the street names before you save them to JSON. 
"""


lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        # YOUR CODE HERE
        # First grab the element's tag and call that the node's "type"
        node["type"] = element.tag

        # Loop through the top level keys
        top_level_keys = element.attrib.keys()
        created = {}
        lat = ""
        lon = ""
        for tlk in top_level_keys:
            if tlk == "lat":
                lat = float(element.get(tlk))
            elif tlk == "lon":
                lon = float(element.get(tlk))
            elif tlk in CREATED:
                created[tlk] = element.get(tlk)
            else:
                node[tlk] = element.get(tlk)

        node["created"] = created
        if lat != "" or lon != "":
            node["pos"] = [lat, lon] 
        
        address_info = {}
        # Now loop through tag's
        for tag in element.iter("tag"):
            p = problemchars.match(tag.attrib["k"])
            if p == None:
                k_array = tag.attrib["k"].split(":")
                if k_array[0] == "addr":
                    if len(k_array) == 2: 
                        if is_street_name(tag):
                            val = update_street_name(tag.attrib["v"], st_mapping, dir_mapping)
                        elif is_zip(tag):
                            val = update_zip(tag.attrib["v"], zip_mapping)
                        elif is_state(tag):
                            val = "FL"
                        else:
                            val = tag.attrib["v"]
                        address_info[k_array[1]] = val
                else:
                    node[tag.attrib["k"]] = tag.attrib["v"]
            
        if len(address_info.keys()) > 0:
            node["address"] = address_info
                
        # Now loop through nd's
        node_refs = []
        for nd in element.iter("nd"):
            node_refs.append(nd.attrib["ref"])
        if len(node_refs) > 0:
            node["node_refs"] = node_refs
        
        return node
    else:
        return None

    
def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data


def test():
    # NOTE: if you are running this code on your computer, with a larger dataset, 
    # call the process_map procedure with pretty=False. The pretty=True option adds 
    # additional spaces to the output, making it significantly larger.
    data = process_map('miami_florida.osm', False)
    print len(data)
    pprint.pprint(data[10])
    pprint.pprint(data[-10])
    

if __name__ == "__main__":
    test()

1672263
{'created': {'changeset': '7463675',
             'timestamp': '2011-03-05T14:23:44Z',
             'uid': '369983',
             'user': 'grouper',
             'version': '3'},
 'id': '95946810',
 'pos': [26.0593811, -80.133733],
 'type': 'node'}
{'created': {'changeset': '34254691',
             'timestamp': '2015-09-26T00:11:14Z',
             'uid': '2679756',
             'user': 'MountainAddict',
             'version': '1'},
 'highway': 'service',
 'id': '372411959',
 'node_refs': ['3759672684',
               '3759668981',
               '3759668982',
               '3759668983',
               '3759668984',
               '3759668983',
               '3759672590',
               '3759672685',
               '3759687160',
               '3759672672'],
 'service': 'parking_aisle',
 'type': 'way'}


In [7]:
import os
file_size = os.path.getsize(OSMFILE + ".bz2")
print "Size of zipped OSM file (" + OSMFILE + ".bz2): " + "{0:.1f}".format(float(file_size)/1000000) + " MB"
file_size = os.path.getsize(OSMFILE)
print "Size of unzipped OSM file (" + OSMFILE + "): " + "{0:.1f}".format(float(file_size)/1000000) + " MB"
file_size = os.path.getsize(OSMFILE + ".json")
print "Size of JSON file (" + OSMFILE + ".json): " + "{0:.1f}".format(float(file_size)/1000000) + " MB"

Size of zipped OSM file (miami_florida.osm.bz2): 23.7 MB
Size of unzipped OSM file (miami_florida.osm): 353.9 MB
Size of JSON file (miami_florida.osm.json): 390.4 MB


>mongoimport --host 127.0.0.1:27017 --db examples --collection miami --file miami_florida.json

In [8]:
# Establish a Pymongo connection with the MongoDB client
from IPython.display import Image
def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db
db = get_db("openstreetmap")

In [9]:
nodes = db.miami.find({"type":"node"}).count()
print "Number of nodes: " + "{:,}".format(nodes)
ways = db.miami.find({"type":"way"}).count()
print "Number of ways: " + "{:,}".format(ways)

Number of nodes: 1,477,183
Number of ways: 194,130


In [10]:
db.miami.find_one()

{u'_id': ObjectId('560d97699d36f37943cbec8a'),
 u'created': {u'changeset': u'15311293',
  u'timestamp': u'2013-03-10T08:11:36Z',
  u'uid': u'1214881',
  u'user': u'ryandrake',
  u'version': u'4'},
 u'id': u'95946794',
 u'pos': [26.2667982, -80.2219395],
 u'type': u'node'}

In [11]:
pipeline = [
    { "$group" : { "_id": "$created.user"}},
    { "$group" : { "_id":1, "count": {"$sum" : 1}}}

]
agg = db.miami.aggregate(pipeline)
for a in agg:
    print "Number of unique users: " + "{:,}".format(a["count"])

Number of unique users: 1,014


In [12]:
ct = db.miami.count() # Total count of collection
pipeline = [
    { "$group" : { "_id": "$created.user",
                 "count" : { "$sum" : 1 }}},
    { "$sort" : { "count" : -1 }},
    { "$limit" : 10 }
]
agg = db.miami.aggregate(pipeline)
for a in agg:
    pct = float(a["count"])/ct
    print a["_id"] + ": " + "{:,}".format(a["count"]) + ", " + "{:.1%}".format(pct) + " of total"

grouper: 299,770, 17.9% of total
woodpeck_fixbot: 236,866, 14.2% of total
Latze: 137,610, 8.2% of total
freebeer: 78,721, 4.7% of total
carciofo: 72,054, 4.3% of total
bot-mode: 62,272, 3.7% of total
NE2: 59,798, 3.6% of total
westendguy: 49,731, 3.0% of total
Seandebasti: 48,397, 2.9% of total
georafa: 39,831, 2.4% of total


In [13]:
pipeline = [
        {"$group" : {"_id" : "$created.user",
                   "count" : {"$sum" : 1}}},
        {"$match" : { "count" : 1 } },
        {"$group" : { "_id" : "$count",
                    "count" : {"$sum" : 1}}}
    ]
agg = db.miami.aggregate(pipeline)
for a in agg:
    print "Number of users with only one entry: " + str(a["count"])

Number of users with only one entry: 210


In [14]:
pipeline = [
            {"$group" : {"_id" : "$amenity",
                        "count" : { "$sum" : 1 }}},
            {"$sort" : { "count" : -1}},
            {"$limit" : 10}
]
agg = db.miami.aggregate(pipeline)
for a in agg:
    print a["_id"], a["count"]

None 1663407
school 2152
parking 1952
place_of_worship 595
kindergarten 521
restaurant 391
fast_food 361
fuel 327
fire_station 314
police 199


In [15]:
pipeline = [
            {"$group" : {"_id" : "$leisure",
                        "count" : { "$sum" : 1 }}},
            {"$sort" : { "count" : -1}},
            {"$limit" : 10}
]
agg = db.miami.aggregate(pipeline)
for a in agg:
    print a["_id"], a["count"]

None 1668086
pitch 1901
park 1008
swimming_pool 703
golf_course 103
playground 95
sports_centre 73
stadium 56
marina 41
track 39


In [16]:
pipeline = [
            {"$group" : {"_id" : "$address.city",
                        "count" : { "$sum" : 1 }}},
            {"$sort" : { "count" : -1}},
            {"$limit" : 10 }
]
agg = db.miami.aggregate(pipeline)
for a in agg:
    print a["_id"], a["count"]

None 1652418
Weston 18251
Miami 336
Fort Lauderdale 188
Miami Beach 137
Wellington 102
Royal Palm Beach 77
Boca Raton 68
Pelican Lake 57
West Palm Beach 53


In [17]:
pipeline = [
            { "$match" : { "address.city" : "Weston"}},
            {"$group" : {"_id" : "$created.user",
                        "count" : { "$sum" : 1 }}},
            {"$sort" : { "count" : -1}},
            {"$limit" : 25 }
]
agg = db.miami.aggregate(pipeline)
for a in agg:
    print a["_id"], a["count"]

dataabierta 18237
ernestocd 8
Auction123 1
bladdiaz 1
adjuva 1
MountainAddict 1
grouper 1
mentor 1


In [18]:
cg = db.miami.find({"address.city":"Weston"})
for c in cg:
    if "name" in c:
        print c["name"], c["created"]["user"]

Alejandro’s by Alejandro (Hair Salon) mentor
Fix Apple Now grouper
165 bladdiaz
Ultimate Software ernestocd
Ultimate Software ernestocd
Auction123.com Auction123
Falcon Cove Middle School ernestocd
Everglades Elementary School ernestocd
Sawgrass Recreation Park adjuva


In [19]:
import pprint
cg = db.miami.find({"address.city":"Coral Gables"})
for c in cg:
    if "name" in c:
        pprint.pprint(c["name"])

u'University of Miami Police Department'
u'Miami Spine & Posture Clinic'
u'Fritz & Franz Bierhaus'
u"Swensen's"
u'Law Office of Ferdie and Lones Chartered'
u'Biltmore Golf Course  ,'
u'Granada Golf Course  ,'
u'Village of Merrick Park'
u'Villa Capri'
u'Otto G. Richter Library'
u'Holiday Inn Coral Gables'
u'7-Eleven'
u'Coral Gables Art Cinema'
u'Books and Books Bookstore/Cafe'
u'Walsh Tower'
u'Rosborough Tower'
u'Hecht Residential College Commons'
u'Pentland Tower'
u'McDonald Tower'
u'Eaton Residential College'
u'Whitten University Center'
u'Hecht-Stanford Dining Hall'
u'Stanford Residential College Commons'
u'Donna E. Shalala Student Center'
u'Gusman Concert Hall'


In [20]:
pipeline = [
            {"$unwind" : "$pos" },
            {"$group" : {"_id" : "$_id",
                        "lat" : { "$first" : "$pos" }}},
            {"$project" : {"_id":0, "lat":1}},
            {"$group" : {"_id" : "$_id",
                        "minLat" : { "$min" : "$lat"},
                        "maxLat" : { "$max" : "$lat"}}}
]
agg = db.miami.aggregate(pipeline)
for a in agg:
    print "Min Latitude: " + str(a["minLat"])
    print "Max Latitude: " + str(a["maxLat"])
pipeline = [
            {"$unwind" : "$pos" },
            {"$group" : {"_id" : "$_id",
                        "lon" : { "$last" : "$pos" }}},
            {"$project" : {"_id":0, "lon":1}},
            {"$group" : {"_id" : "$_id",
                        "minLon" : { "$min" : "$lon"},
                        "maxLon" : { "$max" : "$lon"}}}
]
agg = db.miami.aggregate(pipeline)
for a in agg:
    print "Min Longitude: " + str(a["minLon"])
    print "Max Longitude: " + str(a["maxLon"])

Min Latitude: 25.2910022
Max Latitude: 26.9119981
Min Longitude: -80.682989
Max Longitude: -79.8064041
